In [2]:
# 환경설정
from dotenv import load_dotenv
load_dotenv()

True

### Chain을 이용한 Simple LLM
1. PromptTemplate
2. LLM
3. OutputParser

In [3]:
# 1. PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ('system', '너는 애니메이션을 잘 아는 챗봇이야. 사용자의 질문에 친절하고 상세하게 답변할 수 있어.'), 
    ('user', '{question}')
])

In [4]:
# 2. Model
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.3
)

In [5]:
# 3. OutputParser (StrOutputParser)
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [6]:
# 4. Chain -> 질의
# question: 어떤 포켓몬이 가장 강해?

chain = prompt_template | model | output_parser
chain.invoke({'question': '어떤 포켓몬이 가장 강해?'})

"포켓몬의 강함은 여러 요소에 따라 달라질 수 있습니다. 일반적으로는 전투 능력, 타입 상성, 기술 조합, 그리고 개별 포켓몬의 스탯(HP, 공격력, 방어력 등)에 따라 달라지죠. \n\n현재까지의 포켓몬 게임에서 가장 강력하다고 평가받는 포켓몬 중 하나는 '아르세우스'입니다. 아르세우스는 모든 타입으로 변환할 수 있는 능력을 가지고 있어, 다양한 상황에서 유리한 전투를 할 수 있습니다. \n\n또한, '메가 레쿠자'나 '프리저'와 같은 전설의 포켓몬들도 매우 강력한 성능을 자랑합니다. 메가 진화가 가능한 포켓몬들은 전투에서 강력한 스탯을 가지게 되므로, 전략적으로 활용할 수 있습니다.\n\n결국, 어떤 포켓몬이 가장 강한지는 상황에 따라 다를 수 있으며, 개인의 플레이 스타일과 전략에 따라 달라질 수 있습니다. 특정 포켓몬의 강함을 평가할 때는 팀 구성과 전투 환경도 고려해야 합니다."

In [7]:
from langchain_core.runnables import RunnableSequence

chain = RunnableSequence(prompt_template, model, output_parser)
chain.invoke({'question': '어떤 포켓몬이 가장 강해?'})

"포켓몬의 강함은 여러 요소에 따라 달라질 수 있습니다. 일반적으로는 전투 능력, 타입 상성, 기술, 그리고 전략 등이 중요한 역할을 합니다. \n\n현재까지의 포켓몬 게임에서 가장 강력하다고 평가받는 포켓몬 중 하나는 **아르세우스**입니다. 아르세우스는 '신의 포켓몬'으로 불리며, 모든 타입으로 변환할 수 있는 능력을 가지고 있습니다. 이로 인해 다양한 상황에서 유리한 전투를 할 수 있습니다.\n\n또한, **메가 레쿠쟈**와 **프리즘 레쿠쟈**도 매우 강력한 포켓몬으로 알려져 있습니다. 이들은 높은 스탯과 강력한 기술을 보유하고 있어 많은 트레이너들에게 사랑받고 있습니다.\n\n하지만, 포켓몬 배틀에서는 각 포켓몬의 타입과 상대 포켓몬의 조합, 그리고 트레이너의 전략이 승패를 좌우하기 때문에, 어떤 포켓몬이 가장 강한지는 상황에 따라 달라질 수 있습니다. 특정 대회나 메타에서 강력한 포켓몬이 다를 수 있으니, 이를 고려하는 것도 중요합니다!"

### 단계별 Chatbot
- 첫 대화에서 내 이름을 알려주고, 다음 대화에서 내 이름 기억하는지 물어보기!

1. 그냥 Chat
- langchain_openai의 ChatOPenAI
- langchain_core.message의 클래스

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0.5
)

In [ ]:
from langchain_core.messages import HumanMessage

model.invoke([
    HumanMessage('안녕 내 이름은 유경이야')
]).content      # 모델의 응답 중 메시지만 가져올 수 있음

'안녕하세요, 유경님! 만나서 반가워요. 어떻게 도와드릴까요?'

In [ ]:
# 단건으로 질의를 보내기 때문에 단순한 채팅이 됨
model.invoke([
    HumanMessage('내 이름이 뭐라고 했지?')
]).content

'죄송하지만, 당신의 이름은 알 수 없습니다. 제가 이전 대화 내용을 기억하지 못하기 때문에 이름을 알고 있지 않습니다. 당신의 이름을 알려주시면 그에 맞춰 대화할 수 있습니다!'

2. 직전 대화 맥락 유지
- langchain_openai의 ChatOpenAI
- langchain_core.messages의 클래스

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

messages = []

while True:
    prompt = input("👤 ") 
    if prompt == '끝':
        break

    user_message = HumanMessage(content=prompt)
    messages.append(user_message)
    print(f'💁🏻‍♀️ {user_message.content}')

    response = model.invoke(messages)

    assistant_message = AIMessage(content=response.content)
    messages.append(assistant_message)
    print(f'🤖 {assistant_message.content}')


💁🏻‍♀️ 안녕 내 이름은 유경이야
🤖 안녕하세요, 유경님! 만나서 반갑습니다. 어떻게 도와드릴까요?
💁🏻‍♀️ 내 이름이 뭐라구?
🤖 유경님이라고 하셨습니다. 맞나요?
💁🏻‍♀️ 엉 맞아!
🤖 좋아요, 유경님! 오늘은 어떤 이야기를 나누고 싶으신가요?


3. Memory로 대화의 맥락 유지
- langchain_openai의 ChatOpenAI
- langchain_core.messages의 클래스
- langchain_core.runnables의 클래스
- langchain_core.prompts의 클래스

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

class InMemoryHistory(BaseChatMessageHistory):
    def __init__(self):
        self.messages = []

    def add_messages(self, messages):
        self.messages.extend(messages)

    def clear(self):
        self.messages = []

    def __repr__(self):
        return str(self.messages)

store = {}

def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

prompt = ChatPromptTemplate.from_messages([
    ('system', '너는 {skill}을 가진 AI 어시스턴트야.'), 
    MessagesPlaceholder(variable_name='history'),
    ('human', '{query}')
])

model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.5)
chain = prompt | model

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id,
    input_messages_key='query',
    history_messages_key='history'
)

session_id = 'hyg'
skill = 'ESFJ의 성향'

while True:
    query = input("👤 유저: ")
    if query == '끝':
        print("대화를 종료합니다.")
        break

    print(f'💁🏻‍♀️ {query}')

    response = chain_with_history.invoke(
        {'skill': skill, 'query': query},
        config={'configurable': {'session_id': session_id}}
    )
    print(f"🤖 {response.content}")

    


💁🏻‍♀️ 안녕 나는 유경이야!
🤖 안녕하세요, 유경님! 만나서 반가워요! 오늘은 어떤 이야기를 나눠볼까요? 😊
💁🏻‍♀️ 너가 먼저 재밌는 주제를 꺼내줘
🤖 좋아요! 그럼 요즘 사람들 사이에서 인기 있는 주제 중 하나인 '여행'에 대해 이야기해볼까요? 유경님은 어떤 여행지를 가장 가고 싶으신가요? 아니면 기억에 남는 여행 경험이 있다면 공유해주실래요? 🌍✈️
대화를 종료합니다.


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

prompt - ChatPromptTemplate([
    SystemMessage('당신은 몸시 친절하고 상냥한 친근한 챗봇입니다.'),
    MessagesPlaceholder('history'),
    HumanMessage('{query}')
])

model = ChatOpenAI(model_name='gpt-4o-mini', tenoerature=0.5)
chain = prompt | model

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id,
    input_messages_key='query',
    history_messages_key='history'
)

response = chain_with_history.invoke(
        {'query': '내 이름이 뭐라고 했지?'},
        config={'configurable': {'session_id': 'apple'}}
)

# 채팅 내역 뽑아보기
for message in store['apple'].messages:
    if isinstance(message, HumanMessage):        # isinstance: 객체가 특정 클래스의 인스턴스인지 아닌지
        print(f'🍎: {message.content}')
    elif isinstance(message, AIMessage):
        print(f'🤖: {message.content}')